# Allegro ClearML Keras with TensorBoard example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/allegroai/clearml/blob/master/examples/frameworks/keras/jupyter_keras_TB_example.ipynb)

This example introduces ClearML with Keras and TensorBoard functionality, including automatic logging, models, and TensorBoard outputs. You can find more frameworks examples [here](https://github.com/allegroai/clearml/tree/master/examples/frameworks).

Note: This example is based on the Keras `mnist_mlp.py` example.


In [ ]:
# If you don't have ClearML installed then uncomment this line
# !pip install clearml

In [ ]:
!pip install tensorflow>=2.0

### Create a new task.
To create a new Task object, call the `Task.init` method providing it with `project_name` (the project name for the experiment) and `task_name` (the name of the experiment). When `Task.init` executes, a link to the Web UI Results page for the newly generated Task will be printed, and the Task will be updated in real time in the ClearML demo server.

You can read about the `Task` class in the docs [here](https://clear.ml/docs/latest/docs/fundamentals/task).

In [ ]:
import os
import tempfile

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils as np_utils
from clearml import Task

# Start a new task
task = Task.init(project_name="Colab notebooks", task_name="Keras with TensorBoard example")

In [ ]:
# Train a simple deep NN on the MNIST dataset.
# Gets to 98.40% test accuracy after 20 epochs
# (there is *a lot* of margin for parameter tuning).
# 2 seconds per epoch on a K520 GPU.


class TensorBoardImage(keras.callbacks.TensorBoard):
    @staticmethod
    def make_image(tensor):
        from PIL import Image
        import io
        tensor = np.stack((tensor, tensor, tensor), axis=2)
        height, width, channels = tensor.shape
        image = Image.fromarray(tensor)
        output = io.BytesIO()
        image.save(output, format='PNG')
        image_string = output.getvalue()
        output.close()
        return tf.Summary.Image(height=height,
                                width=width,
                                colorspace=channels,
                                encoded_image_string=image_string)

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        super(TensorBoardImage, self).on_epoch_end(epoch, logs)
        images = self.validation_data[0]  # 0 - data; 1 - labels
        img = (255 * images[0].reshape(28, 28)).astype('uint8')

        image = self.make_image(img)
        summary = tf.Summary(value=[tf.Summary.Value(tag='image', image=image)])
        self.writer.add_summary(summary, epoch)


# the data, shuffled and split between train and test sets
nb_classes = 10
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train = X_train.reshape(60000, 784).astype('float32') / 255.
X_test = X_test.reshape(10000, 784).astype('float32') / 255.
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape=(784,)))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Dense(512))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation('softmax'))

model2 = keras.models.Sequential()
model2.add(keras.layers.Dense(512, input_shape=(784,)))
model2.add(keras.layers.Activation('relu'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

In [ ]:
# Advanced: setting model class enumeration and set it for the task
labels = dict(('digit_%d' % i, i) for i in range(10))
task.set_model_label_enumeration(labels)

output_folder = os.path.join(tempfile.gettempdir(), 'keras_example')

board = keras.callbacks.TensorBoard(histogram_freq=1, log_dir=output_folder, write_images=False)
model_store = keras.callbacks.ModelCheckpoint(filepath=os.path.join(output_folder, 'weight.{epoch}.hdf5'))

In [ ]:
# Fit and evaluate the model

history = model.fit(X_train,
                    Y_train,
                    batch_size=128,
                    epochs=6,
                    callbacks=[board, model_store],
                    verbose=1,
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])